In [1]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd
import datetime
import os

In [2]:
centros_propios = pd.read_excel("centros_propios.xlsx",index_col=0).id_pre_prestador.tolist()

In [3]:
cirugias_validas = pd.read_excel("cirugias_validas.xlsx",index_col=0,dtype=str).prestac_pres_prestacion.tolist()
cirugias_validas = str(cirugias_validas).strip("[").strip("]")

In [4]:
fecha = str(datetime.datetime.today() - datetime.timedelta(days=80))[:7]

In [16]:
mayor= "2019-11"
menor = "2019-10"

In [17]:
data = querydbtopandas(f"""select 
                a11.id_pre_prestador  id_pre_prestador,
                a13.exp_mes, 
                max(a15.desc_pre_nombre)  desc_pre_nombre,
                max(a15.vigente)  vigente,
                a12.id_pre_prestador  id_pre_prestador0,
                max(a16.desc_pre_nombre)  desc_pre_nombre0,
                sum(a11.i_ft_importe_prestacion)  RDENES,
                sum(case  when (  i_ft_importe_prestacion *i_Ft_Cantidad <= 0) then 0  when  (i_ft_importe_prestacion *i_Ft_Cantidad <> 0 ) then   i_Ft_Cantidad   end )  CANTIDADORDENES0
from      DBA.ft_ordenes               a11
                join        DBA.d_auto_autorizacion     a12
                  on        (((a11.id_auto_sucursal * 10000000) + a11.i_ft_autorizacion) = ((a12.id_auto_sucursal * 10000000) + a12.id_auto_autorizacion))
                join        DBA.d_exp_expediente a13
                  on        (((a11.id_ofi_oficina * 10000000) + a11.id_exp_expediente) = ((a13.ofi * 10000000) + a13.id_exp_expediente))
                join        DBA.d_pre_prestador   a15
                  on        (a11.id_pre_prestador = a15.id_pre_prestador)
                join        DBA.d_pre_prestador_autoriza a16
                  on        (a12.id_pre_prestador = a16.id_pre_prestador)
                join        DBA.d_pres_prestacion a17
                  on        (a11.id_pres_prestacion = a17.id_pres_prestacion)
where (CONVERT(DATETIME,a13.exp_mes, "mm-YYYY") >= date("{menor}") and CONVERT(DATETIME,a13.exp_mes, "mm-YYYY") < date("{mayor}")
and a15.id_pre_tipo in ('E','P')
and a11.id_nom_nomenclador in (1,11)
and (not a11.id_pre_prestador in (154688))
and (a12.id_pre_prestador <> 0)
and RTRIM(a17.prestac_pres_prestacion) in ({cirugias_validas}))
group by             a11.id_pres_prestacion,
                a11.id_nom_nomenclador,
                a13.exp_mes,
                a11.id_pre_prestador,
                a12.id_pre_prestador""")

In [18]:
data.exp_mes.nunique()

1

In [19]:
for i in data.exp_mes.unique():
    df = data[data["exp_mes"] == i]
    
    #tope minimo de 32 cirugías por mes
    #total = df.groupby("id_pre_prestador")["CANTIDADORDENES0"].sum() 
    #prestadores_conmasde_32 = total[total >= 32].index
    #df = df[df["id_pre_prestador"].isin(prestadores_conmasde_32)]    
    
    
    df_propios = df[df["id_pre_prestador0"].isin(centros_propios)]

    df_no_propios = df[~df["id_pre_prestador0"].isin(centros_propios)]

    propios = df_propios.groupby(["id_pre_prestador", "desc_pre_nombre"])["CANTIDADORDENES0"].sum().reset_index().rename(columns = {"CANTIDADORDENES0": "propios"})

    no_propios = df_no_propios.groupby(["id_pre_prestador", "desc_pre_nombre"])["CANTIDADORDENES0"].sum().reset_index().rename(columns = {"CANTIDADORDENES0": "no_propios"})

    final = pd.merge(propios, no_propios,how="outer")

    final.fillna(0,inplace=True)

    final["total"]  = final["propios"] + final["no_propios"]

    final["porcentaje en propios"] = (final["propios"] /final["total"])

    final = final.round(2)

    final.rename(columns= {"id_pre_prestador": "Prestador", "desc_pre_nombre": "Razón Social", "propios": "Propias", "no_propios": "No propias", "total": "Total general","porcentaje en propios":"% en Propias" },inplace=True)
    
    nombre = i[-4:]  +"-"+  i[:2]
    #final.to_parquet(f'datos_cirugias/{nombre}.gzip',compression='gzip')  

In [21]:
final[final["Total general"] >= 32]

,Prestador,Razón Social,Propias,No propias,Total general,% en Propias
40,20831,"Giménez,Mariano E.",35.0,1.0,36.0,0.97
75,26786,"Dardanelli,Miguel J.",37.0,0.0,37.0,1.00
184,84719,"Rodríguez Albanese,Gustavo",32.0,0.0,32.0,1.00
196,87366,"Laredo,Fernando G.",62.0,4.0,66.0,0.94
284,144974,"Sayago,Gustavo A.",55.0,0.0,55.0,1.00


In [20]:
final.head(50)

,Prestador,Razón Social,Propias,No propias,Total general,% en Propias
0,172,"Pasqualini,Rodolfo S.",15.0,4.0,19.0,0.79
1,346,"Silberman,Andrés A.",0.0,0.0,0.0,NaN
2,423,"Pérez Macri,Silvia C.",4.0,0.0,4.0,1.00
3,438,"Ortega Traverso,Jorge A.",1.0,0.0,1.0,1.00
4,500,"Darino,Susana E.",1.0,0.0,1.0,1.00
5,509,"Montesinos,Manuel R.",1.0,0.0,1.0,1.00
6,632,"Lavarello,Marcelo",3.0,0.0,3.0,1.00
7,696,"Alfano,Edgardo C.",3.0,0.0,3.0,1.00
8,742,"Ortíz,Javier E.",5.0,0.0,5.0,1.00
9,744,"Vigliocco,Guillermo",13.0,0.0,13.0,1.00


In [37]:
fechas = [i for i in os.listdir('datos_cirugias/') if len(i) == 12]
completo = pd.DataFrame()
for i in fechas:
    completo = pd.concat([completo, pd.read_parquet(f'datos_cirugias/{i}')])

In [39]:
completo.Prestador.nunique()

11

In [40]:
pd.Series(completo.Prestador.unique()).to_csv("prestadores_unicos_cirugias.csv")